# Pregunta 1.3 del proyecto de data mining - Analisis de datos

In [1]:
import requests

# URL of the file I want to download
url_csv = "https://www.omie.es/en/file-access-list?parents%5B0%5D=/&parents%5B1%5D=Day-ahead%20Market&parents%5B2%5D=3.%20Curves&dir=Monthly%20files%20with%20aggregate%20supply%20and%20demand%20curves%20of%20Day-ahead%20market%20including%20bid%20units&realdir=curva_pbc_uof"

# Path where I want to save the file
path = "curbas.csv"

# GET request to try to access URL data
response = requests.get(url_csv)

# I make sure the request succeeded
if response.status_code == 200:
    # ... so I write the data in a local file, in the specified path
    with open(path, "wb") as f:
        f.write(response.content)
    print(f"File saved at {path}")
else:
    print(f"Request error code: {response.status_code}")


c:\miniconda\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Fichier téléchargé et sauvegardé sous curbas.csv
